In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems
from src.embeddings import RecsysEmbedding

experiment_name = 'FlatAttentionClickedOnly2LinearLnorm'
device = 'cuda:2'
seed = 123
pkl_path = '../data/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.9.0'

In [3]:
future_mask = torch.ones((3, 5))
future_mask = torch.triu(future_mask, diagonal=1).bool()
future_mask


tensor([[False,  True,  True,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False,  True,  True]])

# Модель

In [4]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)


class AttentionSequencewiseResponseModel(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1, debug=False):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.nheads = nheads
        self.debug = debug
        self.attention= torch.nn.MultiheadAttention(
            self.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.linear = torch.nn.Linear(2* embedding.embedding_dim, embedding.embedding_dim)
        
        
        
        self.out_layer = torch.nn.Sequential(
                torch.nn.LayerNorm(2* embedding.embedding_dim),
                torch.nn.Linear(embedding.embedding_dim * 2, embedding.embedding_dim * 2),
                torch.nn.GELU(),
                torch.nn.Linear(embedding.embedding_dim * 2, output_dim)
        )

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        device = item_embs.device
        
        if self.debug: print('responses', batch['responses'])
        
        # flattening slates into long sequences
        item_embs = item_embs.flatten(1, 2)
        
        # getting user embedding (mean consumed items)
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2)
        
        # adding a user embedding as a 'zero item' to 
        # make predictions if nothing is observed
        keys = torch.cat([
                user_embs[:, None, :],
                item_embs
            ],
            dim = 1
        )
        
        clicked_mask = batch['responses'].flatten(1, 2) > 0
        if self.debug: print(clicked_mask, clicked_mask.shape)
        clicked_mask = torch.cat([
                torch.ones_like(clicked_mask[:, 0, None]).bool(),
                clicked_mask
            ],
            dim=-1
        )
        if self.debug: print('clicked_mask:', clicked_mask, clicked_mask.shape)
        
        clicked_items = [
            keys[i][clicked_mask[i], :]
            for i in range(shp[0])
        ]
        keys = torch.nn.utils.rnn.pad_sequence(
            clicked_items,
            batch_first=True,
            padding_value=float('nan')
        )
        key_padding_mask = keys.isnan().all(-1)
        keys = keys.nan_to_num(0)
        
        if self.debug: print('key', keys.shape, keys, key_padding_mask)
        
        # forbid model looking into future (and into current iteraction)
        # at the end, mask will be (num_heads * bsize, slate_size * sequence_size, max_len_clicked_items)
        future_mask = torch.ones((item_embs.size(-2) + 1, item_embs.size(-2) + 1 ), device=device)
        future_mask = torch.triu(future_mask, diagonal=0)[1:, :].bool()
        if self.debug: print('future_mask', future_mask.shape, future_mask)
        ####### TODOTODOTODO ########
        # change future masl to be slatewise, not sequencewise
        # build it without large matrix generaton if possible
        # chunk previous iteracrtion
        ############################
        
        if self.debug: print('click_mask_repeated', clicked_mask[None, 0, ...].repeat(item_embs.size(-2), 1))
        
        if self.debug: print(
            'first user click & past', 
            future_mask[clicked_mask[None, 0, ...].repeat(item_embs.size(-2), 1)].reshape(item_embs.size(-2), keys.size(-2) )
        )
        
        attn_mask = [
            future_mask[
                clicked_mask[None, i, ...].repeat(item_embs.size(-2), 1)
            ].reshape(item_embs.size(-2), clicked_mask[None, i, ...].sum() ).T
            for i in range(shp[0])
        ]
        if self.debug: print('attn_mask', attn_mask[0].shape, attn_mask[1].shape )
                
        
        attn_mask = torch.nn.utils.rnn.pad_sequence(
            attn_mask,
            batch_first=True,
            padding_value=True
        ).permute(0, 2, 1)
        if self.debug: print('attn_mask_stacked', attn_mask)
        
        if self.debug: print(item_embs, keys, key_padding_mask, attn_mask.repeat_interleave(self.nheads, 0))
        features, attn_map = self.attention(
            item_embs, keys, keys,
            key_padding_mask=key_padding_mask,
            attn_mask = attn_mask.repeat_interleave(self.nheads, 0)
        )
        
        if self.debug: print(shp, item_embs.shape)
        features = torch.cat(
            [
                features.reshape(shp),
                item_embs.reshape(shp)
            ],
            dim = -1
        )
        
        return self.out_layer(features).squeeze(-1)
        if self.debug: print('features', features.shape, features)
        
        return self.out_layer(features).squeeze(-1)

    
    
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

for batch in dummy_loader:
    break
    
model = AttentionSequencewiseResponseModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='explicit', embedding_dim=2),
    output_dim=1
).to('cpu')

model(batch)


3it [00:00, 2321.57it/s]

biulding affinity matrix...


tensor([[[ 0.2185, -0.0839,  0.2184],
         [ 0.2181, -0.0837,  0.2182]],

        [[ 0.0054,  0.0054,  0.0054],
         [ 0.0054,  0.0054,  0.0054]]], grad_fn=<SqueezeBackward1>)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [5]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

for embeddings in ['explicit', 'svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = AttentionSequencewiseResponseModel(
        RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
        output_dim=1
    ).to('cpu')
    _, metrics = train(model, 
       dummy_loader, dummy_loader, dummy_loader, device='cpu', lr=1e-3, num_epochs=5000, dummy=True,
       silent=True,
#        debug=True,
    
    )


3it [00:00, 2709.50it/s]

biulding affinity matrix...

Evaluating FlatAttentionClickedOnly2LinearLnorm with explicit embeddings



train... loss:0.667537271976471:   0%|                                                                                       | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0}


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

...

Val update: epoch: 2 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.48000000000000004
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 

Evaluating FlatAttentionClickedOnly2LinearLnorm with neural embeddings
Test before learning: {'f1': 0.5, 'roc-auc': 0.3333333134651184, 'accuracy': 0.5}


train... loss:0.6762871146202087:   0%|                                                                              | 2/5000 [00:00<17:47,  4.68it/s]

Val update: epoch: 1 |accuracy: 0.5 | f1: 0.5 | auc: 1.0 | treshold: 0.51
Test: accuracy: 0.5 | f1: 0.5 | auc: 1.0 | 


train... loss:0.6762871146202087:   0%|                                                                              | 2/5000 [00:00<29:58,  2.78it/s]

Val update: epoch: 2 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.56
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


# ContentWise

In [6]:
content_wise_results = []
c = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
c_train_loader, c_val_loader, c_test_loader, c_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(c, batch_size=64)
len(c_train_loader), len(c)

(253, 20216)

In [7]:
for batch in c:
    print(batch['slates_item_ids'].shape)
    break

(28, 50)


In [8]:
for embeddings in ['neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = AttentionSequencewiseResponseModel(
        RecsysEmbedding(train_num_items, c_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       c_train_loader, c_val_loader, c_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)
    
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')


Evaluating FlatAttentionClickedOnly2LinearLnorm with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.17295430600643158, 'roc-auc': 0.5320330858230591, 'accuracy': 0.63466477394104}


train... loss:84.3181885778904:   0%|                                                                            | 1/5000 [01:24<117:56:30, 84.94s/it]

Val update: epoch: 0 |accuracy: 0.7507981657981873 | f1: 0.27686962485313416 | auc: 0.6850188970565796 | treshold: 0.13
Test: accuracy: 0.7520033717155457 | f1: 0.2763349115848541 | auc: 0.6870205402374268 | 


...

train... loss:68.88804356753826:   0%|                                                                           | 6/5000 [07:23<102:32:38, 73.92s/it]

Val update: epoch: 5 |accuracy: 0.870126485824585 | f1: 0.3395519554615021 | auc: 0.7468965649604797 | treshold: 0.19
Test: accuracy: 0.8723294734954834 | f1: 0.3379327058792114 | auc: 0.7466886043548584 | 


train... loss:67.77542945742607:   0%|                                                                           | 7/5000 [08:37<102:55:13, 74.21s/it]

Val update: epoch: 6 |accuracy: 0.8879013657569885 | f1: 0.319448322057724 | auc: 0.7505460977554321 | treshold: 0.21000000000000002
Test: accuracy: 0.8896634578704834 | f1: 0.3216301500797272 | auc: 0.7505181431770325 | 


train... loss:64.8458554148674:   1%|▌                                                                          | 36/5000 [44:55<103:15:07, 74.88s/it]


# RL4RS

In [9]:
!ls ../data/rl4rs-dataset/

item_info.csv		rl4rs_dataset_a_sl.csv	rl4rs_dataset_b_sl.csv
rl4rs_dataset_a_rl.csv	rl4rs_dataset_b_rl.csv


In [10]:
rl4rs_results = []
r = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
r_train_loader, r_val_loader, r_test_loader, r_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(r, batch_size=10000)
len(r_train_loader), len(r)

(4, 45942)

In [11]:
for embeddings in ['svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = AttentionSequencewiseResponseModel(
        RecsysEmbedding(train_num_items, r_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       r_train_loader, r_val_loader, r_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')


Evaluating FlatAttentionClickedOnly2LinearLnorm with svd embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.6679242849349976, 'roc-auc': 0.4804536700248718, 'accuracy': 0.5402732491493225}


train... loss:2.7005019187927246:   0%|                                                                         | 1/5000 [02:35<215:18:55, 155.06s/it]

Val update: epoch: 0 |accuracy: 0.6478014588356018 | f1: 0.7859033346176147 | auc: 0.5354908108711243 | treshold: 0.42000000000000004
Test: accuracy: 0.6456776857376099 | f1: 0.7843275666236877 | auc: 0.5423147678375244 | 


...

train... loss:1.4140877723693848:   1%|▊                                                                     | 56/5000 [1:59:00<174:25:11, 127.00s/it]

Val update: epoch: 55 |accuracy: 0.8518357276916504 | f1: 0.8888848423957825 | auc: 0.9143202304840088 | treshold: 0.44
Test: accuracy: 0.8558819890022278 | f1: 0.8915890455245972 | auc: 0.9176409244537354 | 


train... loss:1.473132461309433:   1%|▊                                                                      | 57/5000 [2:01:06<174:21:20, 126.98s/it]

Val update: epoch: 56 |accuracy: 0.8528515696525574 | f1: 0.890461266040802 | auc: 0.9154578447341919 | treshold: 0.37
Test: accuracy: 0.856220543384552 | f1: 0.8925900459289551 | auc: 0.9186960458755493 | 


train... loss:1.3949508666992188:   1%|▊                                                                     | 61/5000 [2:11:34<177:33:32, 129.42s/it]


In [12]:
rl4rs_results

[{'f1': 0.8925900459289551,
  'roc-auc': 0.9186960458755493,
  'accuracy': 0.856220543384552,
  'embeddings': 'svd'}]

In [13]:
rl4rs_results.append(
    {
  'f1': 0.8974264860153198,
  'roc-auc': 0.9311494827270508,
  'accuracy': 0.8648772835731506,
  'embeddings': 'explicit'}
)